In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy.signal
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from BinaryFileUnpack import BinaryFileUnpack
import warnings

In [ ]:
# Function to get decreasing midline of damping function
def midline(t, c, d):
    # c is initial pressure
    # d quantifies the rate of decrease of pressure
    return c*np.exp(-d * t)

# Fitting damping model to data
def damping(t, A, gamma, omega, phi, c, d):
    warnings.filterwarnings('ignore')
    # First term is equation for damped harmonic motion
    # Second term (midline function) accounts for the gradual loss of pressure in the geyser
    return A * np.exp(-gamma*t) * np.cos(omega*t - phi) + midline(t, c, d)

# Identifying and obtaining parameters for periodic motion based off damping function
def periodic_range(func, time, ydata, fs:int, time_range=5, sens_ind=2):
    std_dev = [1, 1, 1, 1, 1]  # Minimum Standard deviation of the parameters
    params = [1, 1, 1, 1, 1]  # Optimal Parameters of damped harmonic motion with pressure loss
    s_opt = 0
    e_opt = 0
    
    start = 0
    max_ind = 0
    done = False
    while not done:        
        try:
            max_prv = max_ind
            max_ind = np.argmax(ydata[sens_ind, int(start*fs):])
            s = max_ind + int(start*fs)
            e = s + int(fs*time_range)
            
            if e > time.shape[0] or start < 0:
                break
            popt, pcov = curve_fit(func, time[0:e-s], ydata[sens_ind, s:e])
        except RuntimeError:  
            # Least-Squares Regression does not work in range,
            # so change window.
            if start < 8:
                start += 0.5
            else:
                time_range +=1 
        else:
            # Harmonic motion detected
            std_dev_n = np.sqrt(np.diag(pcov))
            
            # Min std_dev
            if std_dev_n[2] < std_dev[2]:
                params = popt
                std_dev = std_dev_n
                s_opt = s
                e_opt = e

            # If the standard deviation is too high, change window
            if ( (2*std_dev_n[2] > 0.005) ):
                # If max point is the same, skip ahead of it
                if max_ind == max_prv:
                    start = (max_ind)/fs*1.1
                else:
                    if start < 8:
                        start += 0.5
                    elif time_range < 8:
                        time_range +=1 
                    else: break
            else:
                done = True

    return params, std_dev, s_opt/fs, e_opt/fs

In [ ]:
import os

# Keep track of the directory with relevant .bin files
parent_dir = ""
os.chdir(parent_dir)

# Name of directory that holds data
# os.chdir()

# Get all the subdirectories in the directory
dirs_raw = []
for file in os.listdir():
    if file.find('.') < 0:
        dirs_raw.append(file)

# Directory names must be in the format: "X{number}{'cm' or 'in'}"
# Directory names must be in the format: "X{number}{'cm' or 'in'}"
dirs = []
for dir in dirs_raw:
    if dir[0] == 'X':
        unit = dir[-2:] 
        if unit == 'cm' or unit == 'in':
            dirs.append(dir)

# List that stores all the test data, seperated by commas (,), to be dumped into .csv
# Header information for data string.
test_summary = ['X (cm),Y (cm),Adjusted Y (cm),start,end,A,damp_factor,omega,phi,c,d,stdev-A,stdev-damp,stdev-omega,stdev-phi,stdev-c,stdev-d']

lowpass = False  # True if lowpass filter is implemented
verbose = True  # True if verobose output and graphs is needed
for dir in dirs:
    os.chdir(dir)
    for file in os.listdir():
        try: name, ext = file.split('.')
        except ValueError: pass  # Not a file
        else: 
            if ext == 'bin':  # Analyze .bin files
                # File format is 'DAQ_{dir}_Y{y_value}-{date}-{time}.bin'
                # Use str.split accordingly to get y_value
                x_value = name.split('-')[0].split('_')[ 1][1:].replace('p', '.')
                y_value = name.split('-')[0].split('_')[-1][1:].replace('p', '.')

                x = float(x_value[:-2])
                y = float(y_value[:-2])

                if x_value[-2:] == 'in':
                    x *= 2.54

                if y_value[-2:] == 'in':
                    y *= 2.54

                if verbose: print(f"For {x_value}, Y{y_value}:")
                
                # Get file details with BinaryFileUnpack class
                obj = BinaryFileUnpack(file)
                fs = int(obj.fs)
                
                # Look at data for sensor 3
                sens_ind = 0
                data = obj.P
                if lowpass:
                    # Implement a lowpass filter
                    bound_freq = 2.2
                    nyq_freq = obj.fs // 2
                    b, a = scipy.signal.butter(3, bound_freq/nyq_freq, 'lowpass')
                    
                    filteredP = np.empty(obj.P.shape)
                    for i in range(filteredP.shape[0]):
                        filteredP[i] = scipy.signal.filtfilt(b, a, obj.P[i])
                    data = filteredP

                popt, std_dev, start, end = periodic_range(damping, obj.time[::1], data[:, ::1], fs, sens_ind=sens_ind)
                
                # Use average value eq on midline func to find average Y
                popt_mid = popt[-2:]
                pres = 1
                pres_std = 0
                if sens_ind == 0:
                    # Use extracted values for sensor 1 calibration
                    df_cal = pd.read_csv(f"{parent_dir}/../../sensor1_calibration.csv")
                    pres = df_cal.iloc[0][0] + x*df_cal.iloc[1][0]
                    # print(pres)
                    pres_std = np.sqrt(df_cal.iloc[0][1]**2 + x*df_cal.iloc[1][1]**2)

                y_avg = (1 / (end - start) * integrate.quad(lambda t: midline(t, *popt_mid), 0, end - start)[0] - pres) * 1e5/98

                # Getting data string for the csv file
                data_string = f"{x},{y},{y_avg+3},{start},{end},{','.join([str(param) for param in popt])},{','.join([str(sd) for sd in std_dev])}"
                test_summary.append(data_string)

                if verbose: 
                    print(f"\tDuration: {start} to {end}\n\tomega = {popt[2]}\n\tstd_dev = {std_dev[2]}\n\t")
                    type_data = "noisy"
                    if lowpass: type_data = "filtered"
                    s = int(start * fs)
                    e = int(end * fs)
                    fig, ax = plt.subplots(1, 1, figsize=(8, 4), tight_layout=True)
                    ax.plot(obj.time[:e-s], data[sens_ind, s:e], color='steelblue', label=f'{type_data} data')
                    ax.plot(obj.time[:e-s], damping(obj.time[:e-s], *popt), color='lime', alpha=0.6, label=f'{type_data} fitted curve')
                    ax.set_xlabel('Time (s)')
                    ax.set_ylabel('Pressure (bar)')
                    ax.set_title(f"X = {x} cm, Y = {y_avg.round(2)} cm")
                    ax.legend()
                    fig.show()
                
    os.chdir('../')

os.chdir(parent_dir)

In [44]:
# Writing to .csv file
# Write initial data
csv_name = 'cold-water-freq-2in-orig.csv'
if lowpass:
    csv_name = 'cold-water-freq-2in-filtered.csv'
with open(csv_name, 'w') as file_write:
    for test in test_summary:
        file_write.write(test + '\n')

# Algebraically alter params to make an equivalent form
# such that params have physical interpretation.
df = pd.read_csv(csv_name)
for i, data in df.iterrows():
    if data[7] < 0:
        df.iloc[i][7] *= -1
    if df.iloc[i][5] < 0:
        df.iloc[i][5] *= -1
        if df.iloc[i][8] < 0:
            df.iloc[i][8] += np.pi
        else:
            df.iloc[i][8] -= np.pi

# Rewrite new form to the csv
df.to_csv(csv_name, index=False)